<a href="https://colab.research.google.com/github/PatrykCiwinski/Chatbot_Openai_XML/blob/main/Chat_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install pinecone-client
!pip install openai

In [ ]:
!pip install nest_asyncio

In [ ]:
!pip install tiktoken

##API keys

In [ ]:
import os

In [ ]:
os.environ['OPENAI_API_KEY'] = '*****'

In [ ]:
import pinecone
pinecone.init(
    api_key='*****',
    environment='****'
)

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


##Scraping the website ( we need sitemap.xml).
### I can't find dataedo.com sitemap, so I used realpython.com. Once it provided we can filter data drom dataedo.com

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from langchain.document_loaders.sitemap import SitemapLoader

loader = SitemapLoader(
    'https://realpython.com/sitemap.xml'
)
docs = loader.load()

Fetching pages: 100%|##########| 4063/4063 [01:54<00:00, 35.58it/s]


##The text is too big. Instead whole text we just use chunks of it.
###We use recommended splitter for the text

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 120,
    length_function = len,
    add_start_index = True,
    separators = ["\n\n","\n",".",",","!","?"," "]
)

doc_chunks = text_splitter.split_documents(docs)

##Then is needed to create emmbedings ( we use openai embeddings)

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

##We use pinecone to store our vectors

In [ ]:
from langchain.vectorstores import Pinecone

index_name = 'first'

#new index creation. It should be created only first time!
# docsearch = Pinecone.from_documents(doc_chunks,embedding=embeddings,index_name=index_name)

#if already have index, just load it

docsearch = Pinecone.from_existing_index(index_name=index_name,embedding=embeddings)

Vectorstore is ready, try first query ( it uses similarity)

In [ ]:
query = 'how to use SQL in python'
docs= docsearch.similarity_search(query)

In [ ]:
print(docs)

[Document(page_content='Python and MongoDB: Connecting to NoSQL Databases\n\n\n\n\ndatabases\nintermediate\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPython and MySQL Database: A Practical Introduction\n\n\n\n\ndatabases\ndata-science\nintermediate\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHandling SQL Databases With PyQt: The Basics\n\n\n\n\ndatabases\ngui\nintermediate\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nData Management With Python, SQLite, and SQLAlchemy\n\n\n\n\ndatabases\nintermediate\nweb-dev\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nIntroduction to Python SQL Libraries\n\n\n\n\nbasics\ndatabases\ntools\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nData Engineer Interview Questions With Python\n\n\n\n\ndatabases\ndevops\nintermediate\n\n\n\n\n\n\n\n\n\n\n\n\n\n\npandas: How to Read and Write Files\n\n\n\n\ndatabases\ndata-science\nintermediate\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPreventing SQL Injection Attacks With Python\n\n\n\n\nbest-practices\ndatabases\nintermediate\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDigging Deeper Into Django Migrations\n\n\n\n\ndat

## But we want answer out of input text
### we need to provide question answer chain

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

In [ ]:
llm = OpenAI()

In [ ]:
from langchain.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=docsearch.as_retriever(),
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},return_source_documents=True)

In [ ]:
query1 = 'how to use SQL in python'

result = qa_chain({"query": query1})

In [ ]:
print(result['source_documents'])

##This approach provides chat functionality with buffered memory

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
from langchain.chains import ConversationalRetrievalChain
retriever=docsearch.as_retriever()
chat = ConversationalRetrievalChain.from_llm(llm,retriever=retriever,memory=memory)

In [ ]:
result = chat({"question": 'excel in medical analysis'})

In [ ]:
result['answer']